- `print()`는 느리다. 입력이 많을것으로 예상되는 경우, `input=sys.stdin.readline`을 사용한다.
  - 메모리가 문제가 되지 않는 경우 `input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline`을 사용할 수 있다.
    - standard input으로 입력 파일의 크기(`os.fstat(0).st_size`)만큼 까지 file descriptor에서 직접 bytes로 읽어들인다. 로우 레벨에서 작동하기에 더 빠르다.
    - 읽어들인 모든 input이 메모리에 저장되므로 주의.
    - `~.readline`은 메모리에 있는 파일에서 한줄만큼의 스트림을 읽어서 반환한다.
    - `sys.stdin.readline`과 달리 개행문자를 포함한 한줄을 bytes형태로 불러온다.
- 전역변수에 접근하는 시간은 로컬변수에 접근하는 시간보다 유의미하게 느리다.
  - 따라서 가능한 경우, 전역 범위에 변수를 선언하는것보단, \
  문제를 푸는 함수(`def sol(): ...`)를 선언해서 그 TC 안에서 필요한 변수들을 사용하는 것이 좋다.
- 파이썬의 index 접근은 생각보다 비효율적이다.
  - 예를들어 list `l`을 index인 `i`로 접근하려 할 때, \
  `for i in range(len(l)): l[i]`보다, `for v in l: v`가 더 빠르며, \
  index가 필요할 경우 `for i, v in enumerate(l): i, v`를 사용하는 것이 더 빠르다.
  - 하지만 list의 길이를 동적으로 두어 append 하는 것(`l.append(x)`)보다, \
  i가 충분히 클 경우 l의 길이를 fixed 해놓고, `l[i] = x`를 사용하는 것이 더 빠르다.
    - https://www.acmicpc.net/source/61132478 `l[i] = x` 구현. 492ms
    - https://www.acmicpc.net/source/61132338 `l.append(x)` 구현. 540ms
  - DP에 저장할 값을 tuple이나 list로 하는 것은 피해야 한다. 9019(DSLR) 참고
    - 값이 2개 이상 필요한 경우 DP 테이블을 하나 더 만들어서 각각 저장하는 것이 더 빠르다.
  - `+` 연산자로 문자열을 합치는 것은 매우 오래 걸린다.
- Python의 String은 immutable 하기 때문에, 합쳐진 문자열을 만들기 위해선 이전 문자열의 모든 문자를 복사하는 작업을 해야 하므로 O(n^2)만큼의 시간이 소요.
- list(deque)나 tuple형태로 저장하고 꺼내쓰는 방식은 매우 비효율적이고, 느립니다.
  - Python은 추가 메모리 할당을 최소화 하기 위해 메모리를 넉넉하게 잡아둔다. 반복문으로 많은 원소를 append를 하게 되면 결국 오래 걸리는 추가 메모리 할당을 해야 하며, 이는 런타임에 진행된다.
    - Python이 메모리를 필요 이상으로 사용하는 것도 이러한 이유
  - list를 만드는 작업 자체도 오래 걸린다. immutable한 값을 그대로 넣는 것보다, 오래걸리는 메모리 할당을 필요로 하는 게 당연히 시간이 더 걸린다.
  - Garbage Collector도 문제가 된다. q에서 list가 빠져나왔을 때, GC는 빠져나온 list를 할당 해제 할지의 여부를 런타임에 확인한다.
    - 즉, (확인하는 시간) + (할당 해제하는 시간)이 런타임에 소모된다.
  - 따라서 가능하면 특히, node는 list로 저장하는 것을 피해야 한다. 분리할 수 있으면 각각 나눠서 처리하자.
